In [296]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
import sys
import warnings
warnings.filterwarnings('ignore')

In [1]:
!pwd

/Users/jseverin/Athena/NREL_github/ATHENA-siem-sumo/Sumo/Simulations


In [297]:
trip = pd.read_csv("tripout_21_Aug_19.csv", sep=";")
stop = pd.read_csv("stop_output.csv", sep=";")

# Cleaning and Processing the Data

In [300]:
bus = stop.loc[stop['stopinfo_type'] == 'bus']
bus.reset_index(inplace=True,drop=True)
bus['time_start'] = bus['stopinfo_started'].apply(lambda x: time.strftime('%H:%M:%S', time.gmtime(int(x))))
bus['time_end'] = bus['stopinfo_ended'].apply(lambda x: time.strftime('%H:%M:%S', time.gmtime(int(x))))
bus['hour_start'] = bus['stopinfo_started'].apply(lambda x: time.strftime('%H', time.gmtime(int(x))))
bus['hour_end'] = bus['stopinfo_ended'].apply(lambda x: time.strftime('%H', time.gmtime(int(x))))
bus['length_duration'] = bus['stopinfo_ended'].values - bus['stopinfo_started'].values
stop_name = []
for name in bus['stopinfo_busStop']:
    if name[0] == "R":
        stop = name[:9]
        stop_name.append(stop)
    else:
        stop = name[0]
        stop_name.append(stop)
bus['stop_name'] = stop_name

bus_names = bus['stopinfo_id'].unique()
dictionary = {}
for b in bus_names:
    tmp = bus.loc[bus['stopinfo_id'] == b]
    
    tmp.reset_index(inplace=True,drop=True)
    
    for i, row in enumerate(tmp['stop_name']):
        try:
            route = row + "_To_" + tmp['stop_name'][i+1]
            route_duration = tmp['stopinfo_ended'][i+1] - tmp['stopinfo_ended'][i]
            hour = tmp['hour_end'][i+1]
            if route not in dictionary:
                dictionary[route] = {"route_duration":[],"hour":[],"terminal":[]}
                dictionary[route]["route_duration"].append(route_duration)
                dictionary[route]['hour'].append(hour)
                dictionary[route]['terminal'].append(b[0])
            else:
                dictionary[route]["route_duration"].append(route_duration)
                dictionary[route]['hour'].append(hour)
                dictionary[route]['terminal'].append(b[0])
        except:
            pass
dframes = {}
for key in dictionary:
    dframes[key] = pd.DataFrame()
    dframes[key]["Travel_time"] = dictionary[key]["route_duration"]
    dframes[key]["Hour"] = dictionary[key]["hour"]
    dframes[key]["terminal"] = dictionary[key]["terminal"]


bus_fuel = trip.loc[trip['tripinfo_vType']=="bus"]
bus_fuel.reset_index(drop=True,inplace=True)
bus_fuel['Terminal'] = bus_fuel['tripinfo_id'].apply(lambda x: x[0])
bus_fuel['gallons'] = bus_fuel['emissions_fuel_abs'].values * 0.000264172
bus_fuel = bus_fuel[['gallons','tripinfo_duration','tripinfo_id','Terminal','tripinfo_routeLength']].groupby('Terminal').mean()
bus_fuel['Terminal'] = bus_fuel.index
bus_fuel.reset_index(inplace=True,drop=True)
number_of_trips = 65
for key in dframes:
    tmp = dframes[key]
    Terminal = dframes[key]['terminal'].unique()[0]
    bus_data = bus_fuel.loc[bus_fuel['Terminal'] == Terminal]
    Total_gallons = bus_data['gallons'].item()
    Duration = bus_data['tripinfo_duration'].item() 
    Distance = bus_data['tripinfo_routeLength'].item() * 0.000621371
    Fuel_usage = []
    distance = []
    
    for time in tmp['Travel_time']:
        percent_of_trip = time/Duration
        Fuel = Total_gallons * percent_of_trip
        Fuel_usage.append(Fuel)
        distance_of_trip = (1/number_of_trips) * Distance
        distance.append(distance_of_trip)
    dframes[key]['Fuel_usage_gallons'] = Fuel_usage
    dframes[key]['Distance_miles'] = distance

# Aggregate the Data and get out what you need

- When you run this you will need to think about what route you want to 
  look at. 
- Here is the function:
    * GrabRouteData(start,end):
    * start options: < "A","B","C","D","E","R" >
    * end options: < "A","B","C","D","E","R" >
         * pointers: "R" is refering to the RAC and the other letters 
         * referring to the other terminals

In [303]:
def GrabRouteData(start,end):
    options = ["A","B","C","D","E","R"]
    if (start not in options) or (end not in options):
         sys.exit("Try again. Use any of these as an option A,B,C,D,E,R")
    if start == "R":
        start = "RAC_pick"
    if end == "R":
        end = "RAC_drop"
    
    route = start + "_To_" +end
    tmp = dframes[route].groupby('Hour').agg(['mean','std'])
    tmp['Hour'] = tmp.index
    tmp.reset_index(inplace=True,drop=True)
       
    
    return tmp
R_A= GrabRouteData("R","A")
R_A.head()

Travel_time           Fuel_usage_gallons           Distance_miles      Hour
         mean       std               mean       std           mean  std     
0  393.045455  3.791138           0.435848  0.004204       8.539649  0.0   00
1  391.520833  2.739564           0.434158  0.003038       8.539649  0.0   01
2  392.250000  3.734970           0.434966  0.004142       8.539649  0.0   02
3  392.000000  3.598611           0.434689  0.003991       8.539649  0.0   03
4  392.750000  2.518680           0.435521  0.002793       8.539649  0.0   04